In [1]:
import numpy as np
import tensorflow as tf
import json
import time
import os
from sklearn.metrics import confusion_matrix, roc_curve, auc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image_dataset_from_directory

def create_model(img_size, num_classes):
    """Create a single EfficientNetB0 model with improved architecture"""
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(*img_size, 3)
    )
    base_model.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.3),  # Increased dropout
        Dense(256, activation='relu'),  # Wider layer
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def create_data_augmentation():
    """Create data augmentation pipeline"""
    return tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomZoom(0.2),
        tf.keras.layers.RandomBrightness(0.2),
        tf.keras.layers.RandomContrast(0.2),
    ])

def ensemble_predict(models, dataset, verbose=True):
    """Make predictions using voting ensemble with detailed logging"""
    # Get individual model predictions
    all_predictions = []
    individual_accuracies = []

    print("\n=== Individual Model Testing ===")
    test_labels = np.concatenate([y.numpy() for x, y in dataset])

    for i, model in enumerate(models):
        # Get model predictions
        predictions = model.predict(dataset, verbose=0)
        class_predictions = np.argmax(predictions, axis=1)
        all_predictions.append(class_predictions)

        # Calculate individual accuracy
        accuracy = np.mean(class_predictions == test_labels)
        individual_accuracies.append(accuracy)
        print(f"Model {i+1} Accuracy: {accuracy:.4f}")

    # Convert to array for voting
    all_predictions = np.array(all_predictions)

    # Implement majority voting
    final_predictions = np.apply_along_axis(
        lambda x: np.bincount(x).argmax(),
        axis=0,
        arr=all_predictions
    )

    # Calculate ensemble accuracy
    ensemble_accuracy = np.mean(final_predictions == test_labels)

    if verbose:
        print("\n=== Ensemble Results ===")
        print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")
        print(f"Average Individual Accuracy: {np.mean(individual_accuracies):.4f}")
        print(f"Best Individual Accuracy: {max(individual_accuracies):.4f}")

    return final_predictions, ensemble_accuracy

def train_and_evaluate(train_dir, test_dir, save_dir, img_size, batch_size=16, num_models=1, epochs=10):
    """Train ensemble and collect metrics with improved logging and performance"""
    # Create save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Load datasets
    print(f"\nLoading datasets for image size {img_size}...")
    train_dataset = image_dataset_from_directory(
        train_dir,
        shuffle=True,
        image_size=img_size,
        batch_size=batch_size
    )

    test_dataset = image_dataset_from_directory(
        test_dir,
        shuffle=False,
        image_size=img_size,
        batch_size=batch_size
    )

    # Get class names
    class_names = train_dataset.class_names
    print(f"Classes: {class_names}")

    # Prepare data for splitting
    print("\nPreparing data splits...")
    train_images = []
    train_labels = []
    for images, labels in train_dataset:
        train_images.append(images.numpy())
        train_labels.append(labels.numpy())
    train_images = np.concatenate(train_images)
    train_labels = np.concatenate(train_labels)

    # Initialize metrics dictionary
    metrics = {
        "image_size": img_size[0],
        "models": [],
        "ensemble_metrics": {},
        "training_time": time.time()
    }

    # Create data augmentation
    data_augmentation = create_data_augmentation()

    # Train individual models
    models = []
    subset_size = len(train_labels) // num_models
    indices = np.random.permutation(len(train_labels))

    print(f"\nTraining {num_models} models...")
    print(f"Total training samples: {len(train_labels)}")
    print(f"Samples per model: {subset_size}")

    for i in range(num_models):
        print(f"\n=== Training Model {i+1}/{num_models} ===")

        # Get subset of data
        start_idx = i * subset_size
        end_idx = start_idx + subset_size
        subset_indices = indices[start_idx:end_idx]

        # Create dataset for this subset
        subset_images = train_images[subset_indices]
        subset_labels = train_labels[subset_indices]

        # Create augmented dataset
        subset_dataset = tf.data.Dataset.from_tensor_slices(
            (subset_images, subset_labels)
        ).shuffle(1000).map(
            lambda x, y: (data_augmentation(x), y),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)

        # Create and train model
        model = create_model(img_size, len(class_names))

        # Add early stopping
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=3,
            restore_best_weights=True
        )

        # Initial training
        print("\nInitial training phase...")
        history = model.fit(
            subset_dataset,
            validation_data=test_dataset,
            epochs=epochs,
            callbacks=[early_stopping],
            verbose=1
        )

        # Fine-tuning
        print("\nFine-tuning phase...")
        model.layers[0].trainable = True
        model.compile(
            optimizer=Adam(learning_rate=1e-5),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        fine_tune_history = model.fit(
            subset_dataset,
            validation_data=test_dataset,
            epochs=5,
            callbacks=[early_stopping],
            verbose=1
        )

        # Collect metrics for this model
        val_accuracy = model.evaluate(test_dataset, verbose=0)[1]
        model_metrics = {
            "model_id": i+1,
            "best_epoch_accuracy": max(history.history['val_accuracy']),
            "final_accuracy": val_accuracy,
            "learning_curves": {
                "accuracy": history.history['accuracy'] + fine_tune_history.history['accuracy'],
                "val_accuracy": history.history['val_accuracy'] + fine_tune_history.history['val_accuracy'],
                "loss": history.history['loss'] + fine_tune_history.history['loss'],
                "val_loss": history.history['val_loss'] + fine_tune_history.history['val_loss']
            }
        }
        metrics["models"].append(model_metrics)
        models.append(model)

    # Ensemble evaluation
    print("\nEvaluating ensemble performance...")
    ensemble_preds, ensemble_accuracy = ensemble_predict(models, test_dataset)
    test_labels = np.concatenate([y.numpy() for x, y in test_dataset])

    # Calculate confusion matrix
    conf_mat = confusion_matrix(test_labels, ensemble_preds)

    # Calculate ROC curves
    roc_curves = []
    for class_idx in range(len(class_names)):
        # Get predictions for this class from all models
        class_probs = np.zeros(len(test_labels))
        for model in models:
            class_probs += model.predict(test_dataset, verbose=0)[:, class_idx]
        class_probs /= len(models)

        # Calculate ROC curve
        fpr, tpr, _ = roc_curve(test_labels == class_idx, class_probs)
        roc_auc = auc(fpr, tpr)
        roc_curves.append({
            "class": class_names[class_idx],
            "fpr": fpr.tolist(),
            "tpr": tpr.tolist(),
            "auc": float(roc_auc)
        })

    # Add ensemble metrics
    metrics["ensemble_metrics"] = {
        "accuracy": float(ensemble_accuracy),
        "confusion_matrix": conf_mat.tolist(),
        "roc_curves": roc_curves
    }

    # Add total training time
    metrics["training_time"] = time.time() - metrics["training_time"]

    # Save metrics
    save_path = os.path.join(save_dir, f"metrics_size_{img_size[0]}.json")
    with open(save_path, 'w') as f:
        json.dump(metrics, f, indent=4)

    print(f"\nMetrics saved to: {save_path}")
    return models, metrics

if __name__ == "__main__":
    # Configuration
    train_dir = '/kaggle/input/dataformachinelearning/model_training/Dhaka Ai Image Dataset with Classification/train'
    test_dir = '/kaggle/input/dataformachinelearning/model_training/Dhaka Ai Image Dataset with Classification/test'
    save_dir = '/kaggle/working/'

    # Image sizes to test
    img_sizes = [(512, 512)]

    # Train and evaluate for each image size
    for img_size in img_sizes:
        print(f"\n{'='*50}")
        print(f"Training ensemble for image size: {img_size}")
        print(f"{'='*50}")

        models, metrics = train_and_evaluate(
            train_dir=train_dir,
            test_dir=test_dir,
            save_dir=save_dir,
            img_size=img_size,
            batch_size=16,  # Adjusted batch size to fit memory constraints
            num_models=3,
            epochs=10
        )

        print(f"\nCompleted training for size {img_size[0]}x{img_size[0]}")
        print(f"Ensemble accuracy: {metrics['ensemble_metrics']['accuracy']:.4f}")
        print(f"{'='*50}\n")



Training ensemble for image size: (512, 512)

Loading datasets for image size (512, 512)...
Found 2390 files belonging to 4 classes.
Found 610 files belonging to 4 classes.
Classes: ['heavy traffic', 'light traffic', 'moderate traffic', 'no traffic']

Preparing data splits...

Training 3 models...
Total training samples: 2390
Samples per model: 796

=== Training Model 1/3 ===
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

Initial training phase...
Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 64s 783ms/step - accuracy: 0.4330 - loss: 1.2462 - val_accuracy: 0.4492 - val_loss: 1.2872
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 383ms/step - accuracy: 0.5758 - loss: 1.0037 - val_accuracy: 0.4180 - val_loss: 1.2153
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 378ms/step - accuracy: 0.6072 - loss: 0.8775 - val_accuracy: 0.4066 - val_loss: 1.1762
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 384ms/step - accuracy: 0.6075 - loss: 0.8780 - val_accuracy: 0.4230 - val_loss: 1.2614

Fine-tuning phase...
Epoc

In [10]:
import numpy as np
import tensorflow as tf
import json
import time
import os
from sklearn.metrics import confusion_matrix, roc_curve, auc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image_dataset_from_directory

def create_model(img_size, num_classes):
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(*img_size, 3)
    )
    base_model.trainable = False
    
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def train_and_save_metrics(train_dir, test_dir, img_size=(512, 512), num_models=3, epochs=10, batch_size=16):
    start_time = time.time()
    
    # Load datasets
    train_dataset = image_dataset_from_directory(
        train_dir,
        shuffle=True,
        image_size=img_size,
        batch_size=batch_size
    )
    
    test_dataset = image_dataset_from_directory(
        test_dir,
        shuffle=False,
        image_size=img_size,
        batch_size=batch_size
    )
    
    class_names = train_dataset.class_names
    
    # Initialize metrics dictionary
    metrics = {
        "image_size": img_size[0],
        "training_time": 0,
        "models": [],
        "ensemble_metrics": {
            "accuracy": 0,
            "confusion_matrix": None,
            "roc_curves": []
        }
    }
    
    # Train models
    models = []
    for i in range(num_models):
        print(f"\nTraining Model {i+1}/{num_models}")
        model = create_model(img_size, len(class_names))
        
        # Train with early stopping
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=3,
            restore_best_weights=True
        )
        
        history = model.fit(
            train_dataset,
            validation_data=test_dataset,
            epochs=epochs,
            callbacks=[early_stopping],
            verbose=1
        )
        
        # Store model metrics
        models.append(model)
        metrics["models"].append({
            "model_id": i+1,
            "best_accuracy": max(history.history['val_accuracy']),
            "learning_curves": {
                "accuracy": history.history['accuracy'],
                "val_accuracy": history.history['val_accuracy'],
                "loss": history.history['loss'],
                "val_loss": history.history['val_loss']
            }
        })
    
    # Calculate ensemble predictions
    test_labels = np.concatenate([y for x, y in test_dataset])
    all_predictions = []
    all_probabilities = []
    
    for model in models:
        probs = model.predict(test_dataset, verbose=0)
        all_probabilities.append(probs)
        all_predictions.append(np.argmax(probs, axis=1))
    
    # Ensemble voting
    ensemble_predictions = np.apply_along_axis(
        lambda x: np.bincount(x).argmax(),
        axis=0,
        arr=np.array(all_predictions)
    )
    
    # Calculate ensemble metrics
    ensemble_accuracy = np.mean(ensemble_predictions == test_labels)
    conf_matrix = confusion_matrix(test_labels, ensemble_predictions)
    
    # Calculate ROC curves
    mean_probabilities = np.mean(all_probabilities, axis=0)
    roc_curves = []
    for i in range(len(class_names)):
        fpr, tpr, _ = roc_curve(test_labels == i, mean_probabilities[:, i])
        roc_auc = auc(fpr, tpr)
        roc_curves.append({
            "class": class_names[i],
            "fpr": fpr.tolist(),
            "tpr": tpr.tolist(),
            "auc": float(roc_auc)
        })
    
    # Update final metrics
    metrics["training_time"] = time.time() - start_time
    metrics["ensemble_metrics"].update({
        "accuracy": float(ensemble_accuracy),
        "confusion_matrix": conf_matrix.tolist(),
        "roc_curves": roc_curves
    })
    
    # Save metrics
    save_dir = '/kaggle/working'
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"metrics_size_{img_size[0]}.json")
    
    try:
        with open(save_path, 'w') as f:
            json.dump(metrics, f, indent=4)
        print(f"\nMetrics successfully saved to: {save_path}")
        print(f"File exists: {os.path.exists(save_path)}")
        print(f"File size: {os.path.getsize(save_path)} bytes")
    except Exception as e:
        print(f"Error saving metrics: {str(e)}")
    
    return metrics

if __name__ == "__main__":
    train_dir = '/kaggle/input/dataformachinelearning/model_training/Dhaka Ai Image Dataset with Classification/train'
    test_dir = '/kaggle/input/dataformachinelearning/model_training/Dhaka Ai Image Dataset with Classification/test'
    
    # Train and save metrics
    metrics = train_and_save_metrics(
        train_dir=train_dir,
        test_dir=test_dir,
        img_size=(512, 512),
        num_models=3,
        epochs=10,
        batch_size=16
    )
    
    # Verify saved file
    save_path = os.path.join('/kaggle/working', f"metrics_size_512.json")
    if os.path.exists(save_path):
        print("\nVerifying saved metrics...")
        with open(save_path, 'r') as f:
            loaded_metrics = json.load(f)
        print("Successfully loaded saved metrics!")
        print(f"Training time: {loaded_metrics['training_time']:.2f} seconds")
        print(f"Ensemble accuracy: {loaded_metrics['ensemble_metrics']['accuracy']:.4f}")
    else:
        print(f"\nWarning: Metrics file not found at {save_path}")

Found 2390 files belonging to 4 classes.
Found 610 files belonging to 4 classes.

Training Model 1/3
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 53s 210ms/step - accuracy: 0.5207 - loss: 1.0201 - val_accuracy: 0.5344 - val_loss: 1.2303
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 11s 75ms/step - accuracy: 0.6647 - loss: 0.7564 - val_accuracy: 0.5443 - val_loss: 1.1293
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - accuracy: 0.7081 - loss: 0.6868 - val_accuracy: 0.5770 - val_loss: 1.2578
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 11s 74ms/step - accuracy: 0.7013 - loss: 0.6759 - val_accuracy: 0.5557 - val_loss: 1.2340
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 11s 74ms/step - accuracy: 0.7404 - loss: 0.6203 - val_accuracy: 0.5492 - val_loss: 1.2200
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 11s 75ms/step - accuracy: 0.7605 - loss: 0.6006 - val_accuracy: 0.5721 - val_loss: 1.2400

Training Model 2/3
Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 42s 172